todo/ideas/notes:
- add general parameterized load_if_exists function to utils to replace data loading functions in scrapers
- derived fighter columns for
    - win streaks
    - ufc record
    - total record
    - champion/former champ status
- only pulling weight from fight data, not fighter data to account for people moving around divisions
    - would be nice to list all weights competed at/have missed weight marks, but weigh in weight isn't present on ufcstats

- add conditional update handling/loading from local data for fighter data
- reorg scraper class so that it makes more sense as a data entry point.
    - probably don't need it to check event links on initiating.

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

# IMMEDIATELY CHANGE TO ROOT DIR
# ONLY RUN THIS ONCE/INCLUDED CODE TO RESTART KERNEL ON EXECUTE.
# POSSIBLY ONLY USE THIS CONTEXT MANAGER
import os
os.chdir('..')
print(os.getcwd())

# local imports once you're in the root
from src.ufctools.scraping import FightDataScraper


d:\code\frankie_edgar_stan_zone


In [2]:
# data scraped as of 5/30
scraper=FightDataScraper()


Pulling event data from http://ufcstats.com/statistics/events/completed?page=all
Reading local event data from d:\code\frankie_edgar_stan_zone\data\event_data.csv
No new events, local data up to date
Loading local fight links from d:\code\frankie_edgar_stan_zone\data\fight_links.pickle
Checking for new events to scrape
No new event links to scrape.
Reading local raw fight data from d:\code\frankie_edgar_stan_zone\data\raw_fight_data.csv


In [3]:
scraper.update_fighter_data()

Getting data for 2572 fighters.


  0%|          | 0/2572 [00:00<?, ?it/s]

KeyboardInterrupt: 